In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
sentences = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are great pets",
    "dog is better than cat",
    "the mat is on the floor"
]

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index)+1
print("Total words : ", total_words)

Total words :  18


In [4]:
def create_cbow(sentences,window_size=2):
    input_data = []
    output_data = []
    
    for sentence in sentences:
        words = sentence.split()
        for i,word in enumerate(words):
            start_index = max(0,i-window_size)
            end_index = min(len(words),i+window_size+1)
            context = [words[j] for j in range(start_index,end_index) if j!=i]
            
            input_data.append(context)
            output_data.append(word)
            
    return input_data, output_data            

In [5]:
input_data,output_data = create_cbow(sentences)
print(input_data)

[['cat', 'sat'], ['the', 'sat', 'on'], ['the', 'cat', 'on', 'the'], ['cat', 'sat', 'the', 'mat'], ['sat', 'on', 'mat'], ['on', 'the'], ['dog', 'sat'], ['the', 'sat', 'on'], ['the', 'dog', 'on', 'the'], ['dog', 'sat', 'the', 'log'], ['sat', 'on', 'log'], ['on', 'the'], ['and', 'dogs'], ['cats', 'dogs', 'are'], ['cats', 'and', 'are', 'great'], ['and', 'dogs', 'great', 'pets'], ['dogs', 'are', 'pets'], ['are', 'great'], ['is', 'better'], ['dog', 'better', 'than'], ['dog', 'is', 'than', 'cat'], ['is', 'better', 'cat'], ['better', 'than'], ['mat', 'is'], ['the', 'is', 'on'], ['the', 'mat', 'on', 'the'], ['mat', 'is', 'the', 'floor'], ['is', 'on', 'floor'], ['on', 'the']]


In [6]:
print(output_data)

['the', 'cat', 'sat', 'on', 'the', 'mat', 'the', 'dog', 'sat', 'on', 'the', 'log', 'cats', 'and', 'dogs', 'are', 'great', 'pets', 'dog', 'is', 'better', 'than', 'cat', 'the', 'mat', 'is', 'on', 'the', 'floor']


In [7]:
input_sequence = tokenizer.texts_to_sequences(input_data)
output_sequence = tokenizer.texts_to_sequences(output_data)
print(input_sequence)

[[3, 4], [1, 4, 2], [1, 3, 2, 1], [3, 4, 1, 5], [4, 2, 5], [2, 1], [6, 4], [1, 4, 2], [1, 6, 2, 1], [6, 4, 1, 8], [4, 2, 8], [2, 1], [10, 11], [9, 11, 12], [9, 10, 12, 13], [10, 11, 13, 14], [11, 12, 14], [12, 13], [7, 15], [6, 15, 16], [6, 7, 16, 3], [7, 15, 3], [15, 16], [5, 7], [1, 7, 2], [1, 5, 2, 1], [5, 7, 1, 17], [7, 2, 17], [2, 1]]


In [8]:
print(output_sequence)

[[1], [3], [4], [2], [1], [5], [1], [6], [4], [2], [1], [8], [9], [10], [11], [12], [13], [14], [6], [7], [15], [16], [3], [1], [5], [7], [2], [1], [17]]


In [9]:
output_sequence = tf.keras.utils.to_categorical(output_sequence,num_classes = total_words)

In [10]:
max_length = max(len(seq) for seq in input_sequence)
input_sequence = pad_sequences(input_sequence,maxlen=max_length,padding='post')

In [11]:
model =Sequential()
model.add(Embedding(input_dim=total_words,output_dim=10))
model.add(Flatten())
model.add(Dense(total_words,activation='softmax'))

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(input_sequence,output_sequence,epochs=10,verbose=1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 2.8944
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: 2.8908
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0345 - loss: 2.8873
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.0690 - loss: 2.8837
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.0690 - loss: 2.8802
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.1379 - loss: 2.8766
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.1379 - loss: 2.8730
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.1379 - loss: 2.8695
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.1379 - loss: 2.8659
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.1724 - loss: 2.8623


In [18]:
# Define the prediction function
def prediction_fun(context_words):
    # Convert context words to sequence and pad
    context_seq = tokenizer.texts_to_sequences([context_words])
    context_seq = pad_sequences(context_seq, maxlen=max_length, padding='post')
    
    # Predict the target word
    predicted = model.predict(context_seq)
    predicted_word_index = np.argmax(predicted)
    return tokenizer.index_word[predicted_word_index]
   


In [19]:
context_example = ["the", "cat", "on", "the"]
prediction_ans = prediction_fun(context_example)
print("Predicted word:", prediction_ans)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Predicted word: sat
